<!-- NOTEBOOK_METADATA source: "⚠️ Jupyter Notebook" title: "Monitor BytePlus model inference with Langfuse" sidebarTitle: "BytePlus" logo: "/images/integrations/byteplus_icon.png" description: "Learn how to integrate BytePlus with Langfuse using the OpenAI drop-in replacement." category: "Integrations" -->

# Observability for BytePlus with Langfuse

This guide shows you how to integrate BytePlus with Langfuse. BytePlus API endpoints for chat, language and code, images, and embeddings are fully compatible with OpenAI's API. This allows us to use the Langfuse OpenAI drop-in replacement to trace all parts of your application.

> **What is BytePlus?** [BytePlus](https://www.byteplus.com/) is a suite of AI-powered APIs and services developed by [ByteDance](https://www.bytedance.com/en/), including speech, video, and recommendation technologies. Langfuse integrates with BytePlus to trace and evaluate LLM workflows that use BytePlus tools, enabling observability across generation and user interaction.

> **What is Langfuse?** [Langfuse](https://langfuse.com) is an open source LLM engineering platform that helps teams trace API calls, monitor performance, and debug issues in their AI applications.

<!-- CALLOUT_START type: "info" emoji: "ℹ️" -->
_**Note:** You can also use BytePlus models in the Langfuse Playground and for LLM-as-a-Jude evaluations using the OpenAI adapter. Find out how to set up an LLM Connection in Langfuse [here](https://langfuse.com/faq/all/llm-connection)._
<!-- CALLOUT_END -->

<!-- STEPS_START -->
## Step 1: Install Dependencies

Make sure you have installed the necessary Python packages:

In [ ]:
%pip install openai langfuse

## Step 2: Set Up Environment Variables

Next, set up your Langfuse API keys. You can get these keys by signing up for a free [Langfuse Cloud](https://cloud.langfuse.com/) account or by [self-hosting Langfuse](https://langfuse.com/self-hosting). These environment variables are essential for the [OpenAI drop-in replacement](https://langfuse.com/integrations/model-providers/openai-py) to authenticate and send data to your Langfuse project.

Find a guide on creating your BytePlus API keys for model services [here](https://docs.byteplus.com/en/docs/ModelArk/1399008). 

In [ ]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

# Get your BytePlus API key from the project settings page
os.environ["ARK_API_KEY"] = "***"

## Step 3: Langfuse OpenAI drop-in Replacement

In this step we use the native [OpenAI drop-in replacement](https://langfuse.com/integrations/model-providers/openai-py) by importing `from langfuse.openai import openai`.

To start using BytePlus models with OpenAI's client libraries, pass in your BytepPlus API key to the `api_key` option, and change the `base_url` to `https://ark.ap-southeast.bytepluses.com/api/v3`:

In [ ]:
# instead of import openai:
from langfuse.openai import openai

client = openai.OpenAI(
  api_key=os.environ.get("ARK_API_KEY"),
  base_url="https://ark.ap-southeast.bytepluses.com/api/v3",
)

**Note:** The OpenAI drop-in replacement is fully compatible with the [Low-Level Langfuse Python SDKs](https://langfuse.com/docs/sdk/python/low-level-sdk) and [`@observe()` decorator](https://langfuse.com/docs/sdk/python/decorators) to trace all parts of your application.

## Step 4: Run An Example

The following cell demonstrates how to call the [Kimi K2 model](https://moonshotai.github.io/Kimi-K2/) via BytePlus using the traced OpenAI client. All API calls will be automatically traced by Langfuse.

In [ ]:
# Non-streaming:
print("----- standard request -----")
completion = client.chat.completions.create(
    # Specify the Ark Inference Point ID that you created, which has been changed for you here to your Endpoint ID
    model="kimi-k2-250711",
    messages=[
        {"role": "system", "content": "You're an AI assistant"},
        {"role": "user", "content": "What is Langfuse?"},
    ],
    name = "BytePlus-Generation" # Optional: Set the name of the generation in Langfuse
)
print(completion.choices[0].message.content)

In [ ]:
# Streaming:
print("----- streaming request -----")
stream = client.chat.completions.create(
    # Specify the Ark Inference Point ID that you created, which has been changed for you here to your Endpoint ID
    model="kimi-k2-250711",
    messages=[
        {"role": "system", "content": "You're an AI assistant"},
        {"role": "user", "content": "What is Langfuse?"},
    ],
    name = "BytePlus-Generation", # Optional: Set the name of the generation in Langfuse
    # Whether the response content is streamed back
    stream=True,
)
for chunk in stream:
    if not chunk.choices:
        continue
    print(chunk.choices[0].delta.content, end="")
print()

## Step 5: See Traces in Langfuse

After running the example model call, you can see the traces in Langfuse. You will see detailed information about your BytePlus API calls, including:

- Request parameters (model, messages, temperature, etc.)
- Response content
- Token usage statistics
- Latency metrics

![Langfuse Trace Example](https://langfuse.com/images/cookbook/integration_byteplus/byteplus-trace.png)

_[Public example trace link in Langfuse](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/b526e1fb8e94dd1e8a17bda69e596199?display=details%3Ftimestamp%3D2025-08-01T09%3A32%3A36.190Z&observation=31a9697edf561af3)_
<!-- STEPS_END -->

<!-- MARKDOWN_COMPONENT name: "LearnMore" path: "@/components-mdx/integration-learn-more.mdx" -->